## Tonale Winter School on Cosmology 2022 
# Tutorial on Gravitational Lensing

## Exercise 6: Strong Lensing mass modeling
In this Notebook, you will learn how to model galaxies and galaxy clusters using strong lensing observations.

>**Warning**: it is important that you execute the cells following the order given in the Notebook. If you execute a cell changing the value of a variable and then go back to some previous cell in the Notebook, the variable value will not change!

We begin by importing some useful packages. We also import the module `lensmodels`.

In [ ]:
# import some useful packages and 

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np 
from astropy.cosmology import FlatLambdaCDM
fontsize = 15 # set the font size for labels in the plots
co = FlatLambdaCDM(H0=70.0, Om0=0.3)
from lensmodels import *

### Strong Lensing
In a strong Lensing event the observables that we can use to model the lens are of three types:
1. Positional constraints: The location of the multiple images of a given source allow us to probe the deflection field, i.e. the first derivatives of the lensing potential;
2. Flux ratios: The flux or area ratios between the multiple images give us information about the magnification, i.e. the second derivatives of the lensing potential. Note that we don't know the intrinsic source flux, thus we can only probe the magnifications relative to one refernce image;
3. Time delays: If the source is intrinsically variable (as supernovae, or some quasars), then the time delay between the images can be measured.

In most cases, the source is not variable (for example in the case of galaxy-galaxy lensing events). In addition, the flux ratios are genereally difficult to reproduce. Indeed, they are very sensitive to perturbers, such as small clumps of matter in the lenses or along the line of sight (actually, anomalous flux ratios is an indication that some clump might be present). Thus, in the vast majority of cases, the positional constraints are the most useful piece of information to unveil the lens mass distribution.

In this tutorial we focus on mostly on these constraints and show how parametric lens modeling works.


#### Image plane optimization
In this part, we consider the so called parametric *image plane optimization* approach: we wish to model a strong lensing event describing the lens with a parametric function and the goal is to build a model that minimizes the distances between observed and model predicted multiple images of a given source.

We begin by simulating a strong lensing event. We consider a massive galaxy as lens and a distant quasar as source. In this case, the source can be assumed to be point-like. For speeding up the calculations, we assume a SIE lens with $\theta_c=0$, i.e. the lens is singular-isothermal. This is not inconsistent with observations of strong lensing galaxies, which indicate the combination of dark matter and baryons has density profile $\rho(r)\propto r^{-2}$ in their central regions. The code below will not work with $\theta_c>0$ unless the `setGrid` method is invoked after the creation of every lens instance.

In [ ]:
FOV = 5.0 # the Field-of-View will be FOVxFOV arcsec 
npix= 512 # the number of pixels in the maps will be (npix x npix)

theta = np.linspace(-FOV/2., FOV/2, npix)

zl = 0.65
zs = 1.66

kwargs_truth = {'zl': zl,
          'zs': zs,
          'sigma0': 250.0,
          'q': 0.6,
          'pa': np.deg2rad(120.0),
          'x1': 0.0,
          'x2': 0.0}

lens_truth = sie(co,**kwargs_truth)
lens_truth.setGrid(theta=theta) # in this case we use setGrid because we want to display the lens convergence.

# we define a dictionary with the source redshift and unlensed coordinates. The flux is not relevant.
beta1 = -0.121 
beta2 = 0.08
kwargs_psr_truth = {
    'zs': zs,
    'ys1': beta1,
    'ys2': beta2,
    'flux': 1.0
}

# we create an instance of the point source. 
ps=pointsrc(size=FOV, sizex=None, sizey=None, Npix=npix, gl=lens_truth, fast=True, **kwargs_psr_truth)
# then we find its multiple images 
thetai_1, thetai_2, mui  = ps.xi1, ps.xi2, ps.mui

def plot_orig(showCL_CAU):
    fig,ax = plt.subplots(1,1,figsize=(5,5))
    ax.imshow(lens_truth.ka,origin='lower',vmax=3,cmap='cubehelix_r',extent=[-FOV/2.,FOV/2.,-FOV/2.,FOV/2.])

    ax.scatter(thetai_1,thetai_2,s=mui*5+15)

    if showCL_CAU:
        tancl=lens.tancl()
        radcl=lens.radcl()

        for cl in tancl:
            cau = lens_truth.crit2cau(cl)
            thetac2, thetac1 = cl[:,0], cl[:,1]
            betac2, betac1 = cau[:,0], cau[:,1]
            ax.plot(thetac1,thetac2,'--',color='white')
            ax.plot(betac1,betac2,'-',color='white')

        for cl in radcl:
            cau = lens_truth.crit2cau(cl)
            thetac2, thetac1 = cl[:,0], cl[:,1]
            betac2, betac1 = cau[:,0], cau[:,1]
            ax.plot(thetac1,thetac2,'--',color='white')
            ax.plot(betac1,betac2,'-',color='white')


    ax.set_aspect('equal')
    ax.xaxis.set_tick_params(labelsize=fontsize)
    ax.yaxis.set_tick_params(labelsize=fontsize)
    ax.set_xlabel(r'$\theta_1$',fontsize=fontsize)
    ax.set_ylabel(r'$\theta_2$',fontsize=fontsize)
    return fig,ax

fig,ax=plot_orig(showCL_CAU=False)

print (('Lens type %s, Einst. radius %f') % (lens_truth.lens_type,lens_truth.bsie()))

The blue circles in the figure are the quasar multiple images. Their sizes are proportional to the image magnifications. The lens is displayed at the center. 

For the moment, we assume an uncertainty on the image positions $\sigma_{ima} = 0.05"$.

>**TASK**: You may add some noise to mimic measurement uncertaintes. For example, you could generate it using a Gaussian with zero mean and standard deviation `sig`. Try repeating the fit using a few different values for this parameter.

In [ ]:
sig = 0.01
s1 = np.random.normal(0.0, sig, len(thetai_1))
s2 = np.random.normal(0.0, sig, len(thetai_2))
thetai_obs_1=thetai_1+s1
thetai_obs_2=thetai_2+s2
sigma_ima=np.zeros(len(thetai_1))+0.05

The first step is to write a function that maps the observed image positions onto the source plane using the lens equation:
$$
\beta_{1,i}=\theta_{1,i}-\alpha_{1,i}(\theta_{1,i},\theta_{2,i}) 
$$
$$
\beta_{2,i}=\theta_{2,i}-\alpha_{2,i}(\theta_{1,i},\theta_{2,i}) 
$$

In [ ]:
# function that calculates the source positions of an ensemble of point images
def guess_source(lens_m,thetai_1,thetai_2):
    # calculate the deflection angle at the image positions
    a1, a2 = lens_m.angle(thetai_1,thetai_2)
    # use lens equation to find source positions
    betai_1=thetai_1-a1
    betai_2=thetai_2-a2
    return betai_1, betai_2

# Test code:
betai1,betai2 = guess_source(lens_truth,thetai_1,thetai_2)
print ('Guessed beta1',betai1,'Truth:',kwargs_psr_truth['ys1'])
print ('Guessed beta2',betai2,'Truth:',kwargs_psr_truth['ys2'])

The function `guess_source` takes the image positions $(\theta_{1,i},\theta_{2,i})$ and returns a guessed source position $(\beta_{1,i},\beta_{2,i})$ for each observed image. So, if we observe four images, then

Now, we can start describing the fitting process. Remember: we don't know anything about the lens. Let's make a guess about the lens parameters and use the lens equation to map the images onto the source plane:


In [ ]:
kwargs_model = {'zl': zl,
          'zs': zs,
          'sigma0': 230.0,
          'q': 0.45,
          'pa': np.deg2rad(164.0),
          'x1': 0.0,
          'x2': 0.0}

lens_model = sie(co,**kwargs_model)
lens_model.setGrid(theta)

betai1,betai2 = guess_source(lens_model,thetai_obs_1,thetai_obs_2)

fig,ax=plt.subplots(1,1,figsize=(5,5))
tancl=lens_model.tancl()
radcl=lens_model.radcl()

for cl in tancl:
    cau = lens_model.crit2cau(cl)
    betac2, betac1 = cau[:,0], cau[:,1]
    ax.plot(betac1,betac2,'-',color='black')

for cl in radcl:
    cau = lens_model.crit2cau(cl)
    betac2, betac1 = cau[:,0], cau[:,1]
    ax.plot(betac1,betac2,'-',color='black')

ax.scatter(betai1,betai2,marker='*',color='yellow',edgecolor='black',s=80,label='Guessed source pos.')

best_beta1 = betai1.mean()
best_beta2 = betai2.mean()

ax.scatter(best_beta1,best_beta2,marker='*',color='blue',edgecolor='black',s=80,label='Guessed source pos. (mean)')

ax.plot(kwargs_psr_truth['ys1'],kwargs_psr_truth['ys2'],'o',ms=8,color='red',label='True source pos.',zorder=-1)
ax.set_aspect('equal')
ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.legend(fontsize=fontsize)
ax.xaxis.set_tick_params(labelsize=fontsize)
ax.yaxis.set_tick_params(labelsize=fontsize)
ax.set_xlabel(r'$\beta_1$',fontsize=fontsize)
ax.set_ylabel(r'$\beta_2$',fontsize=fontsize)


As you can see, we found four well separated sources (indicted by yellow stars), because the lens model is not correct. We may assume that the best guess for the source position is the mean position of these four sources (blue star). Let's try to map this star back to the lens plane: 

In [ ]:
kwargs_psr = {
    'zs': zs,
    'ys1': best_beta1,
    'ys2': best_beta2,
    'flux': 1.0
}

# we create an instance of the point source. 
ps=pointsrc(size=FOV, sizex=None, sizey=None, Npix=npix, gl=lens_model, fast=True, **kwargs_psr)
# then we find its multiple images 
thetai_1_guess, thetai_2_guess, mui_guess = ps.xi1, ps.xi2, ps.mui  


fig,ax=plot_orig(showCL_CAU=False)
ax.scatter(thetai_1_guess, thetai_2_guess, s=mui_guess*5+1,color='orange',zorder=20,edgecolor='black')


We can quantify how bad our guess is by defining a cost function that compares the true image positions with those predicted by the model. In computing the cost function, the trickiest part regards the pairing of observed and model predicted positions. In the implementation below, I created two lists of coordinates and searched for unique pairs from the two lists based on the point distances.

In [ ]:
def cost_function(thetai_1, thetai_2, thetai_1_guess, thetai_2_guess):
    # iterate by value
    coords_obs = [(thetai_1[i],thetai_2[i]) for i in range(len(thetai_1))]
    coords_guess = [(thetai_1_guess[i],thetai_2_guess[i]) for i in range(len(thetai_1_guess))]
    cost=[]
    for list1_val in coords_obs:
        # stop when list2 is empty
        if len(coords_guess) == 0:
            break
        # find the closest match
        list2_val = min(coords_guess, key=lambda x:(x[0]-list1_val[0])**2+(x[1]-list1_val[1])**2)
        d = (list1_val[0]-list2_val[0])**2+(list1_val[1]-list2_val[1])**2
        cost.append(d)
        # remove the match from list2
        coords_guess.remove(list2_val)
        ax.plot([list1_val[0],list2_val[0]],[list1_val[1],list2_val[1]],'-',color='blue')
    # if the model predicts fewer images than observed
    # we penalize the solution
    if len(coords_guess) < len(coords_obs):
        idiff = len(coords_guess)-len(coords_obs)
        for i in range(idiff):
            cost.append(100.0)
    return np.array(cost).sum()
    

fig,ax=plot_orig(showCL_CAU=False)
ax.scatter(thetai_1_guess, thetai_2_guess, s=mui_guess*5+1,color='orange',zorder=20,edgecolor='black')
print (('Cost function %.4f') % cost_function(thetai_obs_1, thetai_obs_2, thetai_1_guess, thetai_2_guess))
    

The goal of parametric modeling is to minimize the cost function by varying the model parameters. Looking at the image above, we will try to reduce the size of the blue sticks connecting the observed image positions (blue circles) to the predicted image positions (orange circles). This is the key objective of the image plane optimization.

There are several algorithms that one can use to perform the lens optimization. In this Notebook we use a Non-Linear Least-Squares Minimization package called [`lmfit`](https://lmfit.github.io/lmfit-py/index.html). This package allows building complex fitting models for non-linear least-squares problems. The implementation shown here was obtained by closely following the examples in the package documentation, which can be found at [this link](http://​cars9.​uchicago.​edu/​software/​python/​lmfit_​MinimizerResult/​intro.​html).

We begin by setting up some initial guesses for the model parameters, storing them in a lmfit.Parameter object, including also some plausible ranges where the parameters can vary:

In [ ]:
import lmfit

# initial guesses. For each parameter we indicate:
# - the name ('sigma0')
# - the initial guess (200.)
# - if the parameter is free to vary (True)
# - the range within the values can vary (100,300). The priors are uniform.

p = lmfit.Parameters()
p.add_many(('sigma0', 200.,True, 100,300),('q', 0.5, True, 0.2, 1.0), 
           ('pa', 100.0, True, 80., 150.),('x1', 0.0, False, -1., 1.),('x2', 0.0, False, -1., 1.))

Note that in this example, we are keeping the lens position fixed (for the keys `x1` and `x2` we use the flag `False` to indicate that their values cannot vary).

In the next cell, we re-write the cost function above in a way that can be used by `lmfit` to optimize the model. In particular, the function accepts the `lmfit.Parameters` object as input and creates the lens instance used to guess the image positions. In addition, the cost value is returned in terms of lists of residuals between the observed and guessed image positions (the length of the blue sticks in the previous figure). 

In [ ]:
def residual(p,thetai_1, thetai_2, sigma_ima):

    kwargs_model = {'zl': zl,
          'zs': zs,
          'sigma0': p['sigma0'],
          'q': p['q'],
          'pa': np.deg2rad(p['pa']),
          'x1': p['x1'],
          'x2': p['x2']}
    lens_model = sie(co,**kwargs_model)
    #lens_model.setGrid(theta)

    betai1,betai2 = guess_source(lens_model,thetai_1,thetai_2)
    best_beta1 = betai1.mean()
    best_beta2 = betai2.mean()

    kwargs_psr = {
        'zs': zs,
        'ys1': best_beta1,
        'ys2': best_beta2,
        'flux': 1.0
    }

    # we create an instance of the point source. 
    ps=pointsrc(size=FOV, sizex=None, sizey=None, Npix=npix, gl=lens_model, fast=True, **kwargs_psr)

    #then we find its multiple images 
    thetai_1_guess, thetai_2_guess = ps.xi1, ps.xi2
    #print (len(thetai_1_guess),len(thetai_1))

    # iterate by value
    coords_obs = [(thetai_1[i],thetai_2[i]) for i in range(len(thetai_1))]
    coords_guess = [(thetai_1_guess[i],thetai_2_guess[i]) for i in range(len(thetai_1_guess))]
    cost1=[]
    cost2=[]

    for list1_val in coords_obs:
        # stop when list2 is empty
        if len(coords_guess) == 0:
            break
        # find the closest match
        list2_val = min(coords_guess, key=lambda x:(x[0]-list1_val[0])**2+(x[1]-list1_val[1])**2)
        
        cost1.append(list1_val[0]-list2_val[0])
        cost2.append(list1_val[1]-list2_val[1])
        
        # remove the match from list2
        coords_guess.remove(list2_val)
       
    # if the model predicts fewer images than observed
    # we penalize the solution

    if len(thetai_1_guess) < len(thetai_1):
        idiff = len(thetai_1) - len(thetai_1_guess)
        for i in range(idiff):
            cost1.append(100.0)
            cost2.append(100.0)

    return np.array(cost1)/sigma_ima, np.array(cost2)/sigma_ima

The next step is to minimize the cost function (i.e. the residuals) to fit the data. We do that by using the `lmfit.minimize` function. Several algorithms are available in `lmfit`. Here, we perform the minimization using the [Powell](https://en.wikipedia.org/wiki/Powell%27s_method) optimization algorithm:

In [ ]:
mi_ip = lmfit.minimize(residual, p, method='Powell', args=(thetai_obs_1, thetai_obs_2, sigma_ima))

lmfit.printfuncs.report_fit(mi_ip.params, min_correl=0.5)
print (lmfit.fit_report(mi_ip))

The fit seems to be successful! Let's visualize how the best fit model predicts the observed image positions: 

In [ ]:
kwargs_model_ip = {'zl': zl,
          'zs': zs,
          'sigma0': mi_ip.params['sigma0'],
          'q': mi_ip.params['q'],
          'pa': np.deg2rad(mi_ip.params['pa']),
          'x1': mi_ip.params['x1'],
          'x2': mi_ip.params['x2']}

lens_model_ip = sie(co,**kwargs_model_ip)
#lens_model.setGrid(theta)

betai1,betai2 = guess_source(lens_model_ip,thetai_obs_1,thetai_obs_2)
best_beta1 = betai1.mean()
best_beta2 = betai2.mean()

kwargs_psr = {
    'zs': zs,
    'ys1': best_beta1,
    'ys2': best_beta2,
    'flux': 1.0
}

# we create an instance of the point source. 
ps=pointsrc(size=FOV, sizex=None, sizey=None, Npix=npix, gl=lens_model_ip, fast=True, **kwargs_psr)

#then we find its multiple images 
thetai_1_guess, thetai_2_guess, mui_guess = ps.xi1, ps.xi2, ps.mui

fig,ax=plot_orig(showCL_CAU=False)
ax.scatter(thetai_1_guess, thetai_2_guess, s=mui_guess*5+1,color='orange',zorder=20,edgecolor='black')

print (('Cost function %.4f') % cost_function(thetai_obs_1, thetai_obs_2, thetai_1_guess, thetai_2_guess))

We perform a Bayesian sampling of the posterior probability distribution of the parameters using the ensemble sampler for Markov chain Monte Carlo (MCMC) implemented in the Python package [`emcee`](https://emcee.readthedocs.io/en/stable/). This can be achieved again by using the `lmfit.minimize` function with the `emcee` method.

Note that for doing this, the cost function has been redefined such as to return a float value, i.e. the $\chi^2(p)$, as specified by setting `float_behavior=’chi2’` in the function call:

In [ ]:
def chi2(p,thetai_1, thetai_2, sigma_ima):
    d1,d2=residual(p,thetai_1, thetai_2, sigma_ima)
    return np.sqrt(d1**2+d2**2)

res = lmfit.minimize(chi2, method='emcee', nan_policy='omit', burn=300, steps=2000, nwalkers=100,
                     params=mi_ip.params, float_behavior='chi2', is_weighted=True, progress=True,args=(thetai_obs_1, thetai_obs_2, sigma_ima))

Finally, we use the `corner` package to display the posterior distributions of the model parameters. Such plot gives you a lot of information about the model, including possible degeneracies between the parameters, how well are the parameters constrained, etc.

In [ ]:
import corner
figure = corner.corner(res.flatchain, labels=[r"$\sigma_0$", r"$q$", r"$\varphi$"],
                       truths=[kwargs_truth['sigma0'],kwargs_truth['q'],np.rad2deg(kwargs_truth['pa'])],
                       quantiles=[0.16, 0.84],
                       show_titles=True, title_kwargs={"fontsize": 14}, label_kwargs={"fontsize": 14})
for ax in figure.get_axes():
    ax.tick_params(axis='both', labelsize=12)
#figure.savefig('corner_sie_with_offset.png')

#### Source plane optimization
As we could experience, the image plane optimization can take time. This kind of optimization is often computationally demanding, because it requires to find the solutions of the lens equation at each iteration and for each family of multiple images. Immagine the lens is a galaxy cluster lensing a hundred galaxies simultaneously. Since the lens requires the combination of several mass components and there are many families of multiple images, the Bayesian analysis can take more than a month even with codes that include parallelization!

There is an alternative and faster approach that consists of finding the best combination of model parameters that minimizes the scatter between the predicted source positions obtained by de-lensing each family of multiple images. This process is called **Optimization in the source plane**.

We can implement this optimization very easily by writing the appropriate cost function. First, of all let's make a plot to illustrate the problem. Let's make a guess about the source position using an initial model and repropose a figure shown earlier:


In [ ]:
kwargs_model = {'zl': zl,
          'zs': zs,
          'sigma0': 230.0,
          'q': 0.45,
          'pa': np.deg2rad(164.0),
          'x1': 0.0,
          'x2': 0.0}

lens_model = sie(co,**kwargs_model)
lens_model.setGrid(theta)

betai1,betai2 = guess_source(lens_model,thetai_obs_1,thetai_obs_2)

fig,ax=plt.subplots(1,1,figsize=(5,5))
tancl=lens_model.tancl()
radcl=lens_model.radcl()

for cl in tancl:
    cau = lens_model.crit2cau(cl)
    betac2, betac1 = cau[:,0], cau[:,1]
    ax.plot(betac1,betac2,'-',color='black')

for cl in radcl:
    cau = lens_model.crit2cau(cl)
    betac2, betac1 = cau[:,0], cau[:,1]
    ax.plot(betac1,betac2,'-',color='black')

ax.scatter(betai1,betai2,marker='*',color='yellow',edgecolor='black',s=80,label='Guessed source pos.')

best_beta1 = betai1.mean()
best_beta2 = betai2.mean()

ax.scatter(best_beta1,best_beta2,marker='*',color='blue',edgecolor='black',s=80,label='Guessed source pos. (mean)')

for i in range(len(betai1)):
    ax.plot([betai1[i],best_beta1],[betai2[i],best_beta2],'-',color='blue')

#ax.plot(kwargs_psr['ys1'],kwargs_psr['ys2'],'o',ms=8,color='red',label='True source pos.')
ax.set_aspect('equal')
ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.legend(fontsize=fontsize)
ax.xaxis.set_tick_params(labelsize=fontsize)
ax.yaxis.set_tick_params(labelsize=fontsize)
ax.set_xlabel(r'$\beta_1$',fontsize=fontsize)
ax.set_ylabel(r'$\beta_2$',fontsize=fontsize)

The distances between the best guessed source position and and the predicted position from each individual image are represented by blue sticks. The goal of the source plane optimisation is to make these sticks as short as possible by varying the model parameters. Indeed, the correct lens model should map all multiple images onto the same source.

We could write the cost function as follows:

In [ ]:
def residuals_sp(p,thetai_1, thetai_2, sigma_src):

    kwargs_model = {'zl': zl,
          'zs': zs,
          'sigma0': p['sigma0'],
          'q': p['q'],
          'pa': np.deg2rad(p['pa']),
          'x1': p['x1'],
          'x2': p['x2']}
    lens_model = sie(co,**kwargs_model)
    #lens_model.setGrid(theta)

    betai1,betai2 = guess_source(lens_model,thetai_1,thetai_2)
    best_beta1 = betai1.mean()
    best_beta2 = betai2.mean()

    cost1=[]
    cost2=[]
    for i in range(len(betai1)):
        cost1.append(betai1[i]-best_beta1)
        cost2.append(betai2[i]-best_beta2)

    return np.array(cost1)/sigma_src, np.array(cost2)/sigma_src

Then, we just need to repeat the same steps from the previous example:

In [ ]:
p = lmfit.Parameters()
p.add_many(('sigma0', 200.,True, 100,300),('q', 0.5, True, 0.2, 1.0), 
           ('pa', 100.0, True, 80., 150.),('x1', 0.0, False, -1., 1.),('x2', 0.0, False, -1., 1.))

sigma_src=np.ones(len(thetai_obs_1))*0.05
mi_src = lmfit.minimize(residuals_sp, p, method='Powell', args=(thetai_obs_1, thetai_obs_2, sigma_src))

lmfit.printfuncs.report_fit(mi_src.params, min_correl=0.5)
print (lmfit.fit_report(mi_src))

To perform the Bayesian analysis, we re-define the cost function as before, and call `lmfit.minimize` with the `emcee` method:

In [ ]:
def chi2_sp(p,thetai_1, thetai_2, sigma_src):
    d1,d2=residuals_sp(p,thetai_1, thetai_2, sigma_src)
    return np.sqrt(d1**2+d2**2)

res_src = lmfit.minimize(chi2_sp, method='emcee', nan_policy='omit', burn=300, steps=2000, nwalkers=100,
                     params=mi_src.params, float_behavior='chi2', is_weighted=True, progress=True,args=(thetai_obs_1, thetai_obs_2, sigma_ima))

In [ ]:
figure = corner.corner(res_src.flatchain, labels=[r"$\sigma_0$", r"$q$", r"$\varphi$"],
                       truths=[kwargs_truth['sigma0'],kwargs_truth['q'],np.rad2deg(kwargs_truth['pa'])],
                       quantiles=[0.16, 0.84],
                       show_titles=True, title_kwargs={"fontsize": 14}, label_kwargs={"fontsize": 14})
for ax in figure.get_axes():
    ax.tick_params(axis='both', labelsize=12)

In this idealized example, the optimization on the source and image planes perform similarly. Let see how the best fit model based on source plane optimization reprodues the observed multiple images:

In [ ]:
kwargs_model_sp = {'zl': zl,
          'zs': zs,
          'sigma0': mi_src.params['sigma0'],
          'q': mi_src.params['q'],
          'pa': np.deg2rad(mi_src.params['pa']),
          'x1': mi_src.params['x1'],
          'x2': mi_src.params['x2']}

lens_model_sp = sie(co,**kwargs_model_sp)
#lens_model.setGrid(theta)

betai1,betai2 = guess_source(lens_model_sp,thetai_obs_1,thetai_obs_2)
best_beta1 = betai1.mean()
best_beta2 = betai2.mean()

kwargs_psr = {
    'zs': zs,
    'ys1': best_beta1,
    'ys2': best_beta2,
    'flux': 1.0
}

# we create an instance of the point source. 
ps=pointsrc(size=FOV, sizex=None, sizey=None, Npix=npix, gl=lens_model_sp, fast=True, **kwargs_psr)

#then we find its multiple images 
thetai_1_guess, thetai_2_guess, mui_guess = ps.xi1, ps.xi2, ps.mui

fig,ax=plot_orig(showCL_CAU=False)
ax.scatter(thetai_1_guess, thetai_2_guess, s=mui_guess*5+1,color='orange',zorder=20,edgecolor='black')
print (cost_function(thetai_obs_1, thetai_obs_2, thetai_1_guess, thetai_2_guess))

The lens model allow us to investigate how the matter (dark and baryonic) is distributed inside the lens.

>**TASK**: make a plot comparing the true and the recovered mass distributions of the lens. Start with the convergence and convert it to solar masses using the function `SigmaCrit` shown below.  Measure the total mass within the FOV.

In [ ]:
%matplotlib inline
def SigmaCrit(lens):
    from astropy.constants import c, G
    c2_G_Msun_Mpc = (c**2/G).to(u.Msun/u.Mpc)
    sigma_cr = c2_G_Msun_Mpc/(4*np.pi)*(lens.ds/lens.dl/lens.dls)
    return(sigma_cr)

lens_model_ip.setGrid(theta=theta)
lens_model_sp.setGrid(theta=theta)


sigmacr = SigmaCrit(lens_truth)
print ('Sigma_crit=',sigmacr)


pixel = np.deg2rad(lens_truth.pixel_scale/3600.0)*lens_truth.dl
print ('pixel scale=',pixel)

# to get rid of units:
pixel = pixel.value
sigmacr = sigmacr.value

#
fig,ax = plt.subplots(1,3,figsize=(15,5))


# TODO: COMPLETE THE CODE BELOW: display the convergence maps of the best fit models obtained from the image and source plane optimizations
ax[0].imshow(np.log10(lens_truth.ka),origin='lower',cmap='cubehelix',extent=[-FOV/2.,FOV/2.,-FOV/2.,FOV/2.])
ax[1].imshow(np.log10(...),origin='lower',cmap='cubehelix',extent=[-FOV/2.,FOV/2.,-FOV/2.,FOV/2.])
ax[2].imshow(np.log10(...),origin='lower',cmap='cubehelix',extent=[-FOV/2.,FOV/2.,-FOV/2.,FOV/2.])

for ax_ in ax:
    ax_.xaxis.set_tick_params(labelsize=fontsize)
    ax_.yaxis.set_tick_params(labelsize=fontsize)
    ax_.set_xlabel(r'$\theta_1$',fontsize=fontsize)
    ax_.set_ylabel(r'$\theta_2$',fontsize=fontsize)
plt.tight_layout()

#TODO: COMPLETE THE CODE BELOW: calculate the mass within the FOV (in solar masses)
print (('True mass %10.4e solMass') % (...))
print (('IP opt. %10.4e solMass') % (...))
print (('SP opt. %10.4e solMass') % (...))

Do these images remind anything to you? :-)

![WFI2033](./data/WFI2033.png)

#### Try yourself!

>**TASK**: Try to fit using a SIE model with $\theta_c=0$ the lens system in the Figure below.

![Your next task](./data/mysteriouslens.png)

The coordinates of the multiple images are:

| Image | $\theta_1$ | $\theta_2$ |
|-------|------------|------------|
|   A   | 0.20956365 | 0.97871616 |
|   B   | -0.97871616| -0.20956365|
|   C   | -0.66800501| 0.66800501 |
|   D . | 0.44537728 | -0.44537728|




Answer these questions:

>**TASK**: Suppose you have measured the time delays between the images. How would you implement a cost function that accounts for both the positional constraints and the time delays?

>**TASK**: Consider the lens below. How would you implement a cost function to fit two families of multiple images?

![Your next task](./data/compoundlens.png)